In [38]:
import pandas as pd
import numpy as num
import seaborn as sea
import matplotlib.pyplot as pl
import math
%matplotlib inline


In [28]:
diabetes=pd.read_csv('diabetic_data.csv')
ids=pd.read_csv('IDs_mapping.csv')

In [29]:
diabetes.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

Droping unneccesary identity columns

In [30]:
diabetes.drop(['encounter_id', 'patient_nbr','admission_type_id', 'discharge_disposition_id', 'admission_source_id'],axis=1,inplace=True)

In [31]:
diabetes

,race,gender,age,weight,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),?,1,?,Pediatrics-Endocrinology,41,0,1,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),?,3,?,?,59,0,18,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),?,2,?,?,11,5,13,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),?,2,?,?,44,1,16,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),?,1,?,?,51,0,8,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),?,3,MC,?,51,0,16,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,AfricanAmerican,Female,[80-90),?,5,MC,?,33,3,18,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,Caucasian,Male,[70-80),?,1,MC,?,53,0,9,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,Caucasian,Female,[80-90),?,10,MC,Surgery-General,45,2,21,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [32]:
diabetes.weight.describe()

count     101766
unique        10
top            ?
freq       98569
Name: weight, dtype: object

Since only 2% of entries have information in the weight column, it would be hard to gain any relavent information from it, so we went ahead and dropped it.

In [33]:
diabetes.drop(['weight'],axis=1,inplace=True)

similarly, this column has no information provided and can also be dropped

In [34]:
diabetes.citoglipton.describe()

count     101766
unique         1
top           No
freq      101766
Name: citoglipton, dtype: object

In [35]:
diabetes.drop(['citoglipton'],axis=1,inplace=True)

At this point it is clear that each column will need to be checked for similar amounts of missing data, so a function will be used to make the process smoother

In [37]:
def describe(inputs):
    for items in inputs:
        print(diabetes[items].describe())
describe(diabetes.columns)

count        101766
unique            6
top       Caucasian
freq          76099
Name: race, dtype: object
count     101766
unique         3
top       Female
freq       54708
Name: gender, dtype: object
count      101766
unique         10
top       [70-80)
freq        26068
Name: age, dtype: object
count    101766.000000
mean          4.395987
std           2.985108
min           1.000000
25%           2.000000
50%           4.000000
75%           6.000000
max          14.000000
Name: time_in_hospital, dtype: float64
count     101766
unique        18
top            ?
freq       40256
Name: payer_code, dtype: object
count     101766
unique        73
top            ?
freq       49949
Name: medical_specialty, dtype: object
count    101766.000000
mean         43.095641
std          19.674362
min           1.000000
25%          31.000000
50%          44.000000
75%          57.000000
max         132.000000
Name: num_lab_procedures, dtype: float64
count    101766.000000
mean          1.339730


Out of the columns it is clear that the following colums lack the necessary information to nake notible predictions with: 

max_glu_serum,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,tolbutamide,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,and metformin-pioglitazone

The cutoff of information used was 95% with many columns having less than 100 entries. Each of them will be dropped.

In [43]:
diabetes.drop(['max_glu_serum','repaglinide','nateglinide','chlorpropamide','glimepiride','acetohexamide','tolbutamide','rosiglitazone','acarbose','miglitol','troglitazone','tolazamide','examide','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone'],axis=1,inplace=True)

In [44]:
diabetes

,race,gender,age,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,number_diagnoses,A1Cresult,metformin,glipizide,glyburide,pioglitazone,insulin,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),1,?,Pediatrics-Endocrinology,41,0,1,0,...,1,None,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),3,?,?,59,0,18,0,...,9,None,No,No,No,No,Up,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),2,?,?,11,5,13,2,...,6,None,No,Steady,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),2,?,?,44,1,16,0,...,7,None,No,No,No,No,Up,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,?,?,51,0,8,0,...,5,None,No,Steady,No,No,Steady,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),3,MC,?,51,0,16,0,...,9,>8,Steady,No,No,No,Down,Ch,Yes,>30
101762,AfricanAmerican,Female,[80-90),5,MC,?,33,3,18,0,...,9,None,No,No,No,No,Steady,No,Yes,NO
101763,Caucasian,Male,[70-80),1,MC,?,53,0,9,1,...,13,None,Steady,No,No,No,Down,Ch,Yes,NO
101764,Caucasian,Female,[80-90),10,MC,Surgery-General,45,2,21,0,...,9,None,No,Steady,No,Steady,Up,Ch,Yes,NO


In [58]:
diabetes.dtypes

race                  object
gender                object
age                   object
time_in_hospital       int64
payer_code            object
medical_specialty     object
num_lab_procedures     int64
num_procedures         int64
num_medications        int64
number_outpatient      int64
number_emergency       int64
number_inpatient       int64
diag_1                object
diag_2                object
diag_3                object
number_diagnoses       int64
A1Cresult             object
metformin             object
glipizide             object
glyburide             object
pioglitazone          object
insulin               object
change                object
diabetesMed           object
readmitted            object
dtype: object

Cleaning step 1: Takeing the upperbound of the age group

In [50]:
diabetes.age

0          [0-10)
1         [10-20)
2         [20-30)
3         [30-40)
4         [40-50)
           ...   
101761    [70-80)
101762    [80-90)
101763    [70-80)
101764    [80-90)
101765    [70-80)
Name: age, Length: 101766, dtype: object

A function will be used to take the upper bound for the age column

In [54]:
def upper(entry):
    return entry[-3:-1]
diabetes['age']=list(map(lambda x:upper(x),diabetes['age']))

In [59]:
diabetes['age']=pd.to_numeric(diabetes['age'])

In [60]:
diabetes['age'].describe()

count    101766.000000
mean         68.222491
std          19.017577
min           0.000000
25%          60.000000
50%          70.000000
75%          80.000000
max          90.000000
Name: age, dtype: float64